Jump to python
==

클로저와 데코레이터
--

* 클로저 (closure)
>  - 함수 안에 내부 함수(inner function)를 구현하고, 내부 함수를 리턴하는 함수
>  - 외부 함수는 자신이 가진 변숫값 등을 내부 함수에 전달 가능

In [12]:
# 어떤 수에 항상 3을 곱해 리턴하는 함수

def mul3(n):
    return n*3

In [13]:
# 항상 5를 곱해 리턴하는 함수

def mul5(n):
    return n*5

>>  - 필요 시 마다 mul6( ), mul7( ), mul8( ) .. 함수를 계속 만드는 것은 비효율적

In [14]:
# closure.py

class Mul:
    def __init__(self, m):
        self.m = m

    def mul(self, n):
        return self.m*n
    
if __name__=="__main__":
    mul3 = Mul(3)
    mul5 = Mul(5)

    print(mul3.mul(10))
    print(mul5.mul(10))

30
50


>>  - 클래스 이용해 특정 값을 미리 설정하고, 그 다음부터 mul( ) 메서드 사용하면 원하는 형태로 호출 가능

>>  - \_\_call\_\_ 메서드 이용해 개선 가능
>>>  - \_\_call\_\_ 함수는 클래스의 객체를 호출하게 만들어주는 메서드로, 인스턴스가 호출됐을 때 실행됨
>>>  - \_\_call\_\_ 함수는 Mul 클래스로 만든 객체에 인수를 전달해, 바로 호출할 수 있도록 함
>>>  - 즉, mul3 객체를 mul3(10) 처럼 호출 가능 

In [15]:
# closure.py

class Mul:
    def __init__(self, m):
        self.m = m

    def __call__(self, n):
        return self.m*n
    
if __name__=="__main__":
    mul3 = Mul(3)
    mul5 = Mul(5)

    print(mul3(10))
    print(mul5(10))

30
50


>>  - 외부 함수 mul 안에 내부 함수 wrapper 구현
>>  - 외부 함수는 내부 함수 wrapper 리턴
>>  - mul 함수에서 wrapper 함수 리턴 시, mul 함수 호출 시 인수로 받은 m 값을 wrapper 함수에 저장하여 리턴
>>  - 클래스가 특정한 값을 설정해 객체를 만드는 과정과 유사
>>  - mul 같은 함수를 클로저(closure) 라고 함

In [16]:
# wrapper.py

def mul(m):
    def wrapper(n):
        return m*n
    return wrapper

if __name__=="__main__":
    mul3 = mul(3)
    mul5 = mul(5)

    print(mul3(10))
    print(mul5(10))

30
50


* 데코레이터 (decorator)
>  - 함수를 꾸미는 함수

>  - "함수가 실행됩니다."라는 문자열 출력하는 myfunc 함수

In [17]:
def myfunc():
    print("함수가 실행됩니다.")

>  - 함수의 실행 시간을 측정하려면?
>>  - 함수의 실행 시간? 함수가 시작하는 순간의 시간과 함수가 종료되는 순간의 시간 차이를 구하자

In [18]:
import time

def myfunc():
    start = time.time()
    print("함수가 실행됩니다.")
    end = time.time()
    print("함수 수행시간: %f 초" % (end-start))

myfunc()

함수가 실행됩니다.
함수 수행시간: 0.000027 초


>>  - 실행 시간을 측정해야 하는 함수가 myfunc 말고도 많을 때, 모든 함수에 이런 코드를 적용하는 것은 비효율적
>>>  - 이때 클로저를 이용하자

In [19]:
# decorator.py

import time

def elapsed(original_func):  # 기존 함수를 인수로 받음
    def wrapper():
        start = time.time()
        result = original_func()  # 기존 함수를 수행
        end = time.time()
        print("함수 수행시간: %f 초" % (end-start))  # 기존 함수의 수행시간 출력
        return result  # 기존 함수의 수행 결과 리턴
    return wrapper
    
def myfunc():
    print("함수가 실행됩니다")

decorated_myfunc = elapsed(myfunc)
decorated_myfunc()

함수가 실행됩니다
함수 수행시간: 0.000028 초


>>  - elapsed 함수로 클로저를 만들었음
>>  - elapsed 함수는 함수를 인수로 받음
>>  - 파이썬에서는 함수도 객체이므로 함수 자체를 인수로 전달 가능

>>  - decorated_myfunc = elapsed(myfunc) 로 생성한 decorated_myfunc를 decorated_myfunc()로 실행 시, 실제로는 elapsed 함수 내부의 wrapper 함수가 실행됨
>>  - wrapper 함수는 전달받은 myfunc 함수를 실행하며 실행 시간을 함께 출력

>  - 클로저 이용 시, 기존 함수에 기능을 덧붙이기 매우 편리

>  - 기존 함수를 바꾸지 않고, 기능을 추가할 수 있게 만드는 elapsed 함수와 같은 클로저가 바로 데코레이터(decorator)

>  - 파이썬 데코레이터는 @ 문자 이용해, 함수 위에 적용하여 사용 가능

In [20]:
# decorator.py

import time

def elapsed(original_func):  # 기존 함수 인수로 받음
    def wrapper():
        start = time.time()
        result = original_func()  # 기존 함수 수행
        end = time.time()
        print("함수 수행시간: %f 초" % (end-start))  # 기존 함수의 수행시간 출력
        return result  # 기존 함수의 수행 결과 리턴
    return wrapper

@elapsed
def myfunc():
    print("함수가 실행됩니다.")

'''

decorated_myfunc = elapsed(myfunc)
decorated_mtfunc()

위의 코드는 @elapsed 데코레이터로 인해 더이상 필요하지 않음

'''
myfunc()

함수가 실행됩니다.
함수 수행시간: 0.000027 초


>  - 문자열을 입력받아 출력하도록 myfunc 함수 수정
>>  - 오류 발생

In [22]:
# decorator2.py

import time

def elapsed(original_func):  # 기존 함수 인수로 받음
    def wrapper():
        start = time.time()
        result = original_func()  # 기존 함수 수행
        end = time.time()
        print("함수 수행시간: %f 초" % (end-start))  # 기존 함수의 수행시간 출력
        return result  # 기존 함수의 수행 결과 리턴
    return wrapper

@elapsed
def myfunc(msg):
    print("'%s'을 출력합니다." % msg)

myfunc("You need python") # 출력할 메시지를 myfunc 파라미터로 전달

TypeError: wrapper() takes 0 positional arguments but 1 was given